# Getting Clumps

I will be focusing on only getting clump spectra from SDSS here.

## Preliminary Functions

In [19]:
from astropy.io import fits
from astropy import table
from astroquery.sdss import SDSS
from astropy import coordinates as coords
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mechanize
from io import BytesIO
from astropy.table import Table
from astropy.table import vstack
from astropy import units as u
from astropy.io import ascii
import matplotlib as mpl

In [3]:
mpl.style.use('dark_background')
mpl.rcParams['figure.facecolor'] = 'k'

## Generating Metallicities

Let's use the M91 calibration mentioned in [Kewley & Ellison 2008 (KE08)](http://adsabs.harvard.edu/abs/2008ApJ...681.1183K):

$$12 + \log(O/H)_\text{lower} = 12 - 4.944 + 0.767x + 0.602x^2 - y(0.29+0.332x - 0.331x^2)$$

$$12 + \log(O/H)_\text{upper} = 12 - 2.939 - 0.2x - 0.37x^2 - 0.305 x^3 - 0.0283x^4 - y(0.0047 - 0.221x - 0.102x^2 - 0.0817x^3 - 0.00717x^4)$$

where $$x = \log R_{23} = \log\left[\frac{[\text{OII}\lambda3727 + [\text{OIII}]\lambda4959 + [\text{OIII}]\lambda5007}{\text{H}\beta}\right]$$
and $$y = \log\text{O}_{32} = \log\left[\frac{[\text{OIII}]\lambda4959 + [\text{OIII}]\lambda5007}{[\text{OII}]\lambda3727}\right]$$

In [4]:
def getlogR23(OII3727,OIII4959,OIII5007,HB):
    return np.log10((OII3727 + OIII4959 + OIII5007)/HB)

def getlogO32(OIII4959,OIII5007,OII3727):
    return np.log10((OIII4959 + OIII5007)/OII3727)

def getlogOHlow(x,y):
    return (-4.944 + 0.767*x + 0.602*x**2 - y*(0.29 + 0.332*x - 0.331*x**2))

def getlogOHup(x,y):
    return (-2.939 - 0.2*x - 0.37*x**2 - 0.305*x**3 - 0.0283*x**4 - 
            y*(0.0047 - 0.221*x - 0.102*x**2 - 0.0817*x**3 - 0.00717*x**4))

### M91 Branches

We need to determine if we are using the upper branch or the lower branch. To do this, we need to get $$\log\left(\frac{[\text{NII}]\lambda6584}{[\text{OII}]\lambda3727}\right)$$

In [151]:
def getlogNiiOii(NII6584,OII3727):
    return np.log10(NII6584/OII3727)

If this value is greater than $-1.2$, then we use the upper branch, while if it is smaller, we use the lower branch.

In [152]:
def M91cal(obj):
    
    x = getlogR23(obj['OII3726'],obj['OIII4959'],obj['OIII5007'],obj['HBeta'])
    y = getlogO32(obj['OIII4959'],obj['OIII5007'],obj['OII3726'])
    
    branchCheck = getlogNiiOii(obj['NII6584'],obj['OII3726'])
    
    if branchCheck > -1.2:
        return branchCheck, getlogOHup(x,y)
    else:
        return branchCheck, getlogOHlow(x,y)

## `astroquery` SQL search

In [153]:
def getSpecDataRADEC(RA,DEC):
    pos = coords.SkyCoord(RA,DEC,unit='deg')
    xid = SDSS.query_region(pos,spectro = True,radius = 10*u.arcsec)
    
    if not xid:
        return False, False
    
    if len(xid) > 1:
#         print ('oh no')
        xid = table.unique(xid,keys='objid',keep='last')
    
    sp = SDSS.get_spectra(matches=xid)
    spData = sp[0][1].data
    return xid

In [154]:
def genSQLsearch(xid):
    SQL = "SELECT \
    s.specObjID, g.class, g.subClass, g.z, g.ra, g.dec, g.plate, g.fiberID, g.MJD, \
    s.oii_3726_flux as OII3726, s.oiii_4959_flux as OIII4959,s.oiii_5007_flux as OIII5007, \
    s.h_beta_flux as HBeta,s.h_alpha_flux as HAlpha,s.nii_6584_flux as NII6584, \
    x.logMass as logM \
    FROM galSpecLine s \
    JOIN stellarMassFSPSGranEarlyDust x ON x.specObjID = s.specObjID \
    JOIN specObj g ON g.specObjID = s.specObjID \
    WHERE g.plate = %s AND g.fiberID = %s"%(xid['plate'],xid['fiberID'])
    return SQL

In [202]:
def genSQLsearchNoM(xid):
    SQL = "SELECT \
    s.specObjID, g.class, g.subClass, g.z, g.ra, g.dec, g.plate, g.fiberID, g.MJD, \
    s.oii_3726_flux as OII3726, s.oiii_4959_flux as OIII4959,s.oiii_5007_flux as OIII5007, \
    s.h_beta_flux as HBeta,s.h_alpha_flux as HAlpha,s.nii_6584_flux as NII6584, \
    FROM galSpecLine s \
    JOIN specObj g ON g.specObjID = s.specObjID \
    WHERE g.plate = %s AND g.fiberID = %s"%(xid['plate'],xid['fiberID'])
    return SQL

In [155]:
xid = getSpecDataRADEC(35.901, 0.226)

In [156]:
queries = [genSQLsearch(x) for x in xid]
# queries

In [157]:
results = [SDSS.query_sql(query) for query in queries]

In [170]:
for res in results:
    res['logNiiOii'],res['M91'] = M91cal(res)

In [171]:
rTable = vstack([res for res in results])

In [172]:
rTable

specObjID,class,subClass,z,ra,dec,plate,fiberID,MJD,OII3726,OIII4959,OIII5007,HBeta,HAlpha,NII6584,logM,logNiiOii,M91
int64,bytes6,bytes11,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
457251183338219520,GALAXY,STARFORMING,0.02494591,35.900857,0.22563594,406,494,51876,58.80363,9.176732,32.95971,34.42345,82.91316,13.68797,9.36815356834157,-0.6330650915970252,-3.16440429415275
1697993723230578688,GALAXY,STARBURST,0.02516786,35.89983,0.22379,1508,497,52944,234.3099,195.3318,572.4767,172.3317,556.3665,34.65907,8.52804988017254,-0.8299737334844213,-3.318679650062327
792769910852839424,GALAXY,STARBURST,0.02490221,35.902477,0.22541812,704,496,52205,278.6412,160.2684,487.0634,182.1586,504.6466,29.88788,9.09170763901077,-0.9695502209339546,-3.2939480773845897


In [241]:
def tableGen(RA,DEC):
    xid = getSpecDataRADEC(RA,DEC)
    
    if not xid:
        print('No xid')
        return
    
    queries = [genSQLsearch(x) for x in xid]
    results = [SDSS.query_sql(query) for query in queries]
    for res in results:
        try:
            res['logNiiOii'],res['M91'] = M91cal(res)
        except TypeError:
            print('No Mass calibration')
            res = [SDSS.query_sql(genSQLsearch)]
    rTable = vstack([res for res in results])
    return rTable

In [242]:
xid = getSpecDataRADEC(4.8778,-0.60175)
xid

ra,dec,objid,run,rerun,camcol,field,z,plate,mjd,fiberID,specobjid,run2d,instrument
float64,float64,int64,int64,int64,int64,int64,float64,int64,int64,int64,int64,int64,bytes4
4.87783844174896,-0.601759911564107,1237657190370443316,2728,301,2,445,0.03285039,389,51795,21,437980866312955904,26,SDSS
4.8794753918292,-0.601721079133158,1237657190370443319,2728,301,2,445,0.03292816,1491,52996,38,1678727256727382016,26,SDSS


In [243]:
queries = [genSQLsearch(x) for x in xid]
results = [SDSS.query_sql(query) for query in queries]

In [244]:
radecList = [(44.171,-0.246),(4.8778,-0.60175)]

In [247]:
tableGen(35.901, 0.226)

specObjID,class,subClass,z,ra,dec,plate,fiberID,MJD,OII3726,OIII4959,OIII5007,HBeta,HAlpha,NII6584,logM,logNiiOii,M91
int64,bytes6,bytes11,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
457251183338219520,GALAXY,STARFORMING,0.02494591,35.900857,0.22563594,406,494,51876,58.80363,9.176732,32.95971,34.42345,82.91316,13.68797,9.36815356834157,-0.6330650915970252,-3.16440429415275
1697993723230578688,GALAXY,STARBURST,0.02516786,35.89983,0.22379,1508,497,52944,234.3099,195.3318,572.4767,172.3317,556.3665,34.65907,8.52804988017254,-0.8299737334844213,-3.318679650062327
792769910852839424,GALAXY,STARBURST,0.02490221,35.902477,0.22541812,704,496,52205,278.6412,160.2684,487.0634,182.1586,504.6466,29.88788,9.09170763901077,-0.9695502209339546,-3.2939480773845897
